In [1]:
from sagas.nlu.uni_chunks import get_chunk
from sagas.nlu.ruleset_procs import list_words, cached_chunks
from sagas.conf.conf import cf
# get_chunk(f'verb_domains', 'xcomp/obj', lambda w: w.upos)
sents='What do you think about the war?'
lang='en'
domain='verb'

dn=lambda domain: f'{domain}_domains' if domain!='predicts' else domain
chunks = cached_chunks(sents, lang, cf.engine(lang))
domains=chunks[dn(domain)]
domains

[{'index': 4,
  'text': 'think',
  'lemma': 'think',
  'upos': 'VERB',
  'xpos': 'VB',
  'feats': 'VerbForm=Inf',
  'governor': 0,
  'dependency_relation': 'root',
  'entity': [],
  'segments': [],
  'obl': [{'index': 7,
    'text': 'war',
    'lemma': 'war',
    'upos': 'NOUN',
    'xpos': 'NN',
    'feats': 'Number=Sing',
    'governor': 4,
    'dependency_relation': 'obl',
    'entity': [],
    'segments': [],
    'case': [{'index': 5,
      'text': 'about',
      'lemma': 'about',
      'upos': 'ADP',
      'xpos': 'IN',
      'feats': '_',
      'governor': 7,
      'dependency_relation': 'case',
      'entity': [],
      'segments': []}],
    'det': [{'index': 6,
      'text': 'the',
      'lemma': 'the',
      'upos': 'DET',
      'xpos': 'DT',
      'feats': 'Definite=Def|PronType=Art',
      'governor': 7,
      'dependency_relation': 'det',
      'entity': [],
      'segments': []}]}],
  'aux': [{'index': 2,
    'text': 'do',
    'lemma': 'do',
    'upos': 'AUX',
    'xpos': 

In [2]:
from sagas.nlu.nlu_tools import vis_tree
for ds in domains:
    vis_tree(ds)

root: think(verb, 4)
├── obl: war(noun, 7)
│   ├── case: about(adp, 5)
│   └── det: the(det, 6)
├── aux: do(aux, 2)
├── nsubj: you(pron, 3)
├── punct: ?(punct, 8)
└── obj: what(pron, 1)


In [3]:
result=get_chunk(chunks, dn(domain), 'obj', lambda w: (w.text, w.upos.lower()))
result

[('What', 'pron')]

In [4]:
list(domains[0].keys())

['index',
 'text',
 'lemma',
 'upos',
 'xpos',
 'feats',
 'governor',
 'dependency_relation',
 'entity',
 'segments',
 'obl',
 'aux',
 'nsubj',
 'punct',
 'obj']

In [12]:
from sagas.util.collection_util import to_obj
def flat_table(ds, parent, table_rs, is_root=True):
    child_tags=[k for k,v in ds.items() if isinstance(v, list) and k not in ('entity', 'segments')]
    path=f"{parent}/{ds['dependency_relation']}" if not is_root else ''
    data={}
    for k,v in ds.items():
        if k in child_tags:
            for vchild in v:
                flat_table(vchild, path, table_rs, is_root=False)
        else:
            data[k]=v
    table_rs.append(to_obj({'path':path, **data}))

table_rs=[]
for ds in domains:
    flat_table(ds, '', table_rs)
print([(r.path, r.text) for r in table_rs])
table_rs

[('/obl/case', 'about'), ('/obl/det', 'the'), ('/obl', 'war'), ('/aux', 'do'), ('/nsubj', 'you'), ('/punct', '?'), ('/obj', 'What'), ('', 'think')]


[{'path': '/obl/case',
  'index': 5,
  'text': 'about',
  'lemma': 'about',
  'upos': 'ADP',
  'xpos': 'IN',
  'feats': '_',
  'governor': 7,
  'dependency_relation': 'case',
  'entity': [],
  'segments': []},
 {'path': '/obl/det',
  'index': 6,
  'text': 'the',
  'lemma': 'the',
  'upos': 'DET',
  'xpos': 'DT',
  'feats': 'Definite=Def|PronType=Art',
  'governor': 7,
  'dependency_relation': 'det',
  'entity': [],
  'segments': []},
 {'path': '/obl',
  'index': 7,
  'text': 'war',
  'lemma': 'war',
  'upos': 'NOUN',
  'xpos': 'NN',
  'feats': 'Number=Sing',
  'governor': 4,
  'dependency_relation': 'obl',
  'entity': [],
  'segments': []},
 {'path': '/aux',
  'index': 2,
  'text': 'do',
  'lemma': 'do',
  'upos': 'AUX',
  'xpos': 'VBP',
  'feats': 'Mood=Ind|Tense=Pres|VerbForm=Fin',
  'governor': 4,
  'dependency_relation': 'aux',
  'entity': [],
  'segments': []},
 {'path': '/nsubj',
  'index': 3,
  'text': 'you',
  'lemma': 'you',
  'upos': 'PRON',
  'xpos': 'PRP',
  'feats': 'Case=

In [6]:
import rx
from rx import operators as ops

from dataclasses import dataclass
@dataclass
class token_data:
    word: str
    pos: str
    path: str

source = rx.of(*table_rs)

composed = source.pipe(
    ops.filter(lambda t: t.upos=='VERB'),
    ops.map(lambda t: token_data(word=f"{t.text}/{t.lemma}", 
                                 # pos=t.upos.lower(), 
                                 path=t.path)),    
)
composed.subscribe(lambda value: print("Received {0}".format(value)),
                  on_error=lambda e: print(e),)

__init__() missing 1 required positional argument: 'pos'


In [7]:
def filter_pos(pos):
    return rx.pipe(
        ops.filter(lambda t: t.upos.lower()==pos),
    )
def to_token():
    return rx.pipe(
        ops.map(lambda t: token_data(word=f"{t.text}/{t.lemma}", 
                                     path=t.path,
                                     pos=t.upos.lower())), 
    )
source.pipe(
    filter_pos('verb'),
    to_token(), 
).subscribe(
    on_next=lambda value: print("Received {0}".format(value)),
    on_error=lambda e: print(e),
    on_completed=lambda: print("Process done!"),
)

Received token_data(word='think/think', pos='verb', path='/root')
Process done!


In [8]:
from blinker import NamedSignal, signal
collect = signal('collect')

@collect.connect
def collect_verb(sender, **kwargs):
    results=[]    
    source = rx.of(*kwargs['rs'])
    source.pipe(
        filter_pos('verb'),
        to_token(), 
    ).subscribe(
        on_next=lambda value: results.append(value),
        on_error=lambda e: print(e),
        on_completed=lambda: print("Process done!"),
    )
    return results

# results=collect(source)

results=[]
result = collect.send('sender', rs=table_rs)
results.extend([{'name':fn.__name__, 'result':r} for fn, r in result])
results

Process done!


[{'name': 'collect_verb',
  'result': [token_data(word='think/think', pos='verb', path='/root')]}]

In [9]:
def filter_pos(pos):
    return rx.pipe(
        ops.filter(lambda t: t.upos.lower()==pos),
    )
def to_token():
    return rx.pipe(
        ops.map(lambda t: token_data(word=f"{t.text}/{t.lemma}", 
                                     path=t.path,
                                     pos=t.upos.lower())), 
    )

class Pipeline(object):
    def __init__(self, source, **kwargs):
        self.vars=to_obj(kwargs)
        self.composed=source.pipe(
            filter_pos(self.vars.pos),
            to_token(), 
        )
    def run(self):
        results=[]
        self.composed.subscribe(
            on_next=lambda value: results.append(value),
            on_error=lambda e: print(e),
            on_completed=lambda: print("Process done!"),
        )
        return results

Pipeline(source, pos='verb').run()

Process done!


[token_data(word='think/think', pos='verb', path='/root')]

In [10]:
from dataclasses import dataclass, fields
[fld.name for fld in fields(token_data)]

['word', 'pos', 'path']